In [1]:
from google.colab import files
uploaded = files.upload()


Saving ml-100k.zip to ml-100k (1).zip


In [2]:
import zipfile
with zipfile.ZipFile("ml-100k.zip", 'r') as zip_ref:
    zip_ref.extractall()


In [3]:
import os
print(os.listdir())


['.config', 'ml-100k.zip', 'ml-100k (1).zip', 'ml-100k', 'sample_data']


In [4]:
print(os.listdir("ml-100k"))


['u1.test', 'u3.test', 'u4.base', 'u3.base', 'u4.test', 'allbut.pl', 'u.genre', 'u5.test', 'u.item', 'u.occupation', 'ua.base', 'ub.test', 'README', 'u.info', 'ua.test', 'ub.base', 'u5.base', 'u1.base', 'u.data', 'mku.sh', 'u.user', 'u2.base', 'u2.test']


**Load Ratings Data: Load the u.data file into a DataFrame and name the columns**

In [5]:
import pandas as pd
ratings = pd.read_csv("ml-100k/u.data", sep="\t", names=["user_id", "item_id", "rating", "timestamp"])
ratings.head()


,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


**Create the User-Item Matrix:Create a matrix where each row is a user and each column is a movie**

In [6]:
user_item_matrix = ratings.pivot_table(index='user_id', columns='item_id', values='rating')
user_item_matrix.head()


item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**User-Based Collaborative Filtering:Compute similarity between users using cosine similarity**

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
user_item_filled = user_item_matrix.fillna(0)
user_similarity = cosine_similarity(user_item_filled)
user_similarity_df = pd.DataFrame(user_similarity, index=user_item_matrix.index, columns=user_item_matrix.index)
user_similarity_df.head()


user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.166931,0.047460,0.064358,0.378475,0.430239,0.440367,0.319072,0.078138,0.376544,...,0.369527,0.119482,0.274876,0.189705,0.197326,0.118095,0.314072,0.148617,0.179508,0.398175
2,0.166931,1.000000,0.110591,0.178121,0.072979,0.245843,0.107328,0.103344,0.161048,0.159862,...,0.156986,0.307942,0.358789,0.424046,0.319889,0.228583,0.226790,0.161485,0.172268,0.105798
3,0.047460,0.110591,1.000000,0.344151,0.021245,0.072415,0.066137,0.083060,0.061040,0.065151,...,0.031875,0.042753,0.163829,0.069038,0.124245,0.026271,0.161890,0.101243,0.133416,0.026556
4,0.064358,0.178121,0.344151,1.000000,0.031804,0.068044,0.091230,0.188060,0.101284,0.060859,...,0.052107,0.036784,0.133115,0.193471,0.146058,0.030138,0.196858,0.152041,0.170086,0.058752
5,0.378475,0.072979,0.021245,0.031804,1.000000,0.237286,0.373600,0.248930,0.056847,0.201427,...,0.338794,0.080580,0.094924,0.079779,0.148607,0.071459,0.239955,0.139595,0.152497,0.313941


**Recommend Movies to a User (User-Based):Recommend top-rated unseen movies based on similar users**

In [8]:
target_user_id = 1
similar_users = user_similarity_df[target_user_id].drop(index=target_user_id)

similar_users = similar_users.sort_values(ascending=False)


In [9]:
user_rated_movies = user_item_matrix.loc[target_user_id].dropna().index

top_n = 5

weighted_ratings = pd.Series(dtype='float64')

for other_user_id, similarity_score in similar_users.items():
    other_ratings = user_item_matrix.loc[other_user_id]
    for item_id, rating in other_ratings.dropna().items():
        if item_id not in user_rated_movies:
            weighted_ratings[item_id] = weighted_ratings.get(item_id, 0) + rating * similarity_score

recommended_items = weighted_ratings.sort_values(ascending=False).head(top_n)

recommended_items


,0
318,472.055290
423,433.972006
357,415.627178
286,404.102899
288,393.350465


**Load Movie Titles and Display Recommendations:Merge movie titles using u.item to show recommended movie names**

In [10]:
movie_cols = ['item_id', 'movie_title'] + [f'col{i}' for i in range(22)]
movies = pd.read_csv("ml-100k/u.item", sep='|', encoding='latin-1', names=movie_cols, usecols=[0, 1])

movies.head()


,item_id,movie_title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [11]:
recommended_df = recommended_items.reset_index()
recommended_df.columns = ['item_id', 'score']

final_recommendations = pd.merge(recommended_df, movies, on='item_id')

final_recommendations[['movie_title', 'score']]


,movie_title,score
0,Schindler's List (1993),472.055290
1,E.T. the Extra-Terrestrial (1982),433.972006
2,One Flew Over the Cuckoo's Nest (1975),415.627178
3,"English Patient, The (1996)",404.102899
4,Scream (1996),393.350465


**Evaluate Precision**

In [12]:
target_user_id = 150

liked_movies = ratings[(ratings['user_id'] == target_user_id) & (ratings['rating'] >= 3.5)]['item_id'].tolist()


In [13]:
recommended_movie_ids = final_recommendations['item_id'].tolist()

K = len(recommended_movie_ids)


In [14]:
hits = [movie for movie in recommended_movie_ids if movie in liked_movies]
precision_at_k = len(hits) / K

print(f"Precision@{K} = {precision_at_k:.2f}")


Precision@5 = 0.20


**Item-Based Collaborative Filtering**

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

item_user_matrix = user_item_matrix.T.fillna(0)

item_similarity = cosine_similarity(item_user_matrix)

item_similarity_df = pd.DataFrame(item_similarity, index=item_user_matrix.index, columns=item_user_matrix.index)

item_similarity_df.head()


item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
item_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.402382,0.330245,0.454938,0.286714,0.116344,0.620979,0.481114,0.496288,0.273935,...,0.035387,0.0,0.000000,0.000000,0.035387,0.0,0.0,0.0,0.047183,0.047183
2,0.402382,1.000000,0.273069,0.502571,0.318836,0.083563,0.383403,0.337002,0.255252,0.171082,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.078299,0.078299
3,0.330245,0.273069,1.000000,0.324866,0.212957,0.106722,0.372921,0.200794,0.273669,0.158104,...,0.000000,0.0,0.000000,0.000000,0.032292,0.0,0.0,0.0,0.000000,0.096875
4,0.454938,0.502571,0.324866,1.000000,0.334239,0.090308,0.489283,0.490236,0.419044,0.252561,...,0.000000,0.0,0.094022,0.094022,0.037609,0.0,0.0,0.0,0.056413,0.075218
5,0.286714,0.318836,0.212957,0.334239,1.000000,0.037299,0.334769,0.259161,0.272448,0.055453,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.094211


In [16]:
target_movie_id = 50
similar_movies = item_similarity_df[target_movie_id].drop(index=target_movie_id)
similar_movies = similar_movies.sort_values(ascending=False)

top_similar_movies = similar_movies.head(5)
top_similar_movies_df = pd.DataFrame(top_similar_movies).reset_index()
top_similar_movies_df.columns = ['item_id', 'similarity']

similar_movie_titles = pd.merge(top_similar_movies_df, movies, on='item_id')
similar_movie_titles[['movie_title', 'similarity']]


,movie_title,similarity
0,Return of the Jedi (1983),0.884476
1,Raiders of the Lost Ark (1981),0.764885
2,"Empire Strikes Back, The (1980)",0.749819
3,Toy Story (1995),0.734572
4,"Godfather, The (1972)",0.697332


 Fix NumPy Compatibility & Install Surprise

In [17]:
!pip install numpy==1.24.4

!pip install scikit-surprise --upgrade --no-binary :all:


** Fix NumPy Compatibility & Install Surprise**

In [19]:
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

reader = Reader(line_format='user item rating timestamp', sep='\t')
data = Dataset.load_from_df(ratings[['user_id', 'item_id', 'rating']], reader)

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)


In [20]:
from surprise import SVD
model = SVD()
model.fit(trainset)


** Evaluate SVD Model Performance**

In [21]:
from surprise import accuracy

predictions = model.test(testset)

rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)


RMSE: 0.9361
MAE:  0.7375


**Recommend Top Unseen Movies for a User**

In [22]:
user_id = 1

user_movies = ratings[ratings['user_id'] == user_id]['item_id'].tolist()
all_movies = ratings['item_id'].unique()
unseen_movies = [movie for movie in all_movies if movie not in user_movies]

predicted_scores = [(movie_id, model.predict(user_id, movie_id).est) for movie_id in unseen_movies]

top_recommendations = sorted(predicted_scores, key=lambda x: x[1], reverse=True)[:5]

top_recommendations_df = pd.DataFrame(top_recommendations, columns=['item_id', 'predicted_rating'])
top_recommendations_named = pd.merge(top_recommendations_df, movies, on='item_id')
top_recommendations_named[['movie_title', 'predicted_rating']]


,movie_title,predicted_rating
0,"Close Shave, A (1995)",4.787803
1,Schindler's List (1993),4.688098
2,"Third Man, The (1949)",4.656768
3,His Girl Friday (1940),4.567029
4,Rear Window (1954),4.536524


In [23]:
import pickle
with open("svd_model.pkl", "wb") as f:
    pickle.dump(model, f)
